In [2]:
import SimpleITK as sitk
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

ALPHA = 0.5
BLACK = float(0)
SPACING = 0.5

"""
def load_image_1():
    image_path = "registered_7.nii.gz"
    image = sitk.ReadImage(image_path, sitk.sitkFloat32)
    arr = sitk.GetArrayFromImage(image)

def load_image_2():
    image_path_2 = "registered_11to7.nii.gz"
    image2 = sitk.ReadImage(image_path_2, sitk.sitkFloat32)
    arr2 = sitk.GetArrayFromImage(image2)
"""

def is_image_1(fused_image_coordinates):
    """Voxel coordinates that constitute image 1"""
    # if not in bounds
    if not(max_1_x - 1 > fused_image[0] and max_1_y - 1 > fused_image[1] and max_1_z - 1 > fused_image[2]):
        return False

    # if black
    if arr[fused_image[0], fused_image[1], fused_image[2]] == BLACK:
        return False
    

def is_image_2(fused_image_coordinates):
    """Voxel coordinates that constitute image 2"""
    # if not in bounds
    if not(max_2_x - 1 > fused_image[0] and max_2_y - 1 > fused_image[1] and max_2_z - 1 > fused_image[2]):
        return False

    # if black
    if arr2[fused_image[0], fused_image[1], fused_image[2]] == BLACK:
        return False
    
    return True

def is_both(fused_image_coordinates):
    """Voxel coordinates that constitute both images ie. the overlapping regions"""
    return is_image_1(fused_image_coordinates) and is_image_2(fused_image_coordinates)

def set_point(x, y, z, value):
    fused_image.SetPixel(int(x), int(y), int(z), int(value))

def output_image():
    """Set up fused image"""
    sitk.WriteImage(fused_image, 'fused.nii.gz')
    slice_fused_image = fused_arr[:,200,:]
    plt.imshow(slice_fused_image,'gray')

# load images
image_path = "registered_7.nii.gz"
image = sitk.ReadImage(image_path, sitk.sitkFloat32)
arr = sitk.GetArrayFromImage(image)

image_path_2 = "registered_11to7.nii.gz"
image2 = sitk.ReadImage(image_path_2, sitk.sitkFloat32)
arr2 = sitk.GetArrayFromImage(image2)

# get meta data
size1 = image.GetSize()
origin1 = image.GetOrigin()
spacing1 = image.GetSpacing()

size2 = image2.GetSize()
origin2 = image2.GetOrigin()
spacing2 = image2.GetSpacing()

[max_1_x, max_1_y, max_1_z] = size1
[max_2_x, max_2_y, max_2_z] = size2

# fused image metadata
size_fused = max(image.TransformIndexToPhysicalPoint((size1)), image2.TransformIndexToPhysicalPoint((size2)))
origin_fused = min(image.TransformIndexToPhysicalPoint((int(origin1[0]),int(origin1[1]),int(origin1[2]))),image2.TransformIndexToPhysicalPoint((int(origin2[0]),int(origin2[1]),int(origin2[2]))))
max_fused = max(image.TransformIndexToPhysicalPoint((size1)),image2.TransformIndexToPhysicalPoint((size2)))

# change physical points to indexes
size_fused = image2.TransformPhysicalPointToIndex((size_fused))

# create fused image
fused_image = sitk.Image(size_fused, sitk.sitkFloat32)
fused_image.SetOrigin(origin_fused)
fused_image.SetSpacing((SPACING, SPACING, SPACING))
origin_fused = fused_image.TransformPhysicalPointToIndex((origin_fused))
fused_arr = sitk.GetArrayFromImage(fused_image)
# [fused_image_x, fused_image_y, fused_image_z] = fused_arr

# loop over fused image coordinates
for x in np.arange(origin_fused[0], max_fused[0] - 1):
    for y in np.arange(origin_fused[1], max_fused[1] - 1):
        for z in np.arange(origin_fused[2], max_fused[2] - 1):
            if is_both((x, y, z)):
                #alpha blending
                value = ALPHA * arr[x, y, z] + (1 - ALPHA) * arr2[x, y, z]
                set_point(x, y, z, value)
            elif is_image_1((x, y, z)):
                #get image 1 value
                value = arr[x, y, z]
                set_point(x, y, z, value)
            elif is_image_2((x, y, z)):
                #get image 2 value
                value = arr2[x, y, z]
                set_point(x, y, z, value)

TypeError: in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'